In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_lstm_2025_04_04_11_22.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.000000
1,star_graph_n7,4.0,3.962707
2,star_graph_n7,5.0,5.111387
3,star_graph_n7,0.0,0.000000
4,star_graph_n7,3.0,3.178828


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
1,star_graph_n7,4.0,3.962707,0.037293,0.001391,0.0,0.009323
2,star_graph_n7,5.0,5.111387,0.111387,0.012407,0.0,0.022277
3,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
4,star_graph_n7,3.0,3.178828,0.178828,0.031980,0.0,0.059609
...,...,...,...,...,...,...,...
5275,graph_powerlaw_cluster_graph_n7,3.0,2.928821,0.071179,0.005066,0.0,0.023726
5276,graph_powerlaw_cluster_graph_n7,1.0,1.106576,0.106576,0.011358,0.0,0.106576
5277,graph_powerlaw_cluster_graph_n7,3.0,3.144140,0.144140,0.020776,0.0,0.048047
5278,graph_powerlaw_cluster_graph_n7,1.0,1.051922,0.051922,0.002696,0.0,0.051922


In [9]:
df['RE'].sum()/len(df)

0.06216979097938306

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,2.0,1678,127.225858,0.075820,0.041207
1,3.0,1663,107.560010,0.064678,0.062746
2,1.0,944,61.073417,0.064696,0.007841
3,4.0,693,28.319606,0.040865,0.041932
4,0.0,235,2.104072,0.008953,0.000527
5,5.0,66,1.955401,0.029627,0.039712
6,6.0,1,0.018133,0.018133,0.011837


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,236.638748,0.060568,0.040898
1,graph_powerlaw_cluster_graph_n7,1350,91.188877,0.067547,0.038958
2,star_graph_n7,23,0.428871,0.018647,0.014260


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3785,235.991697,0.062349,0.042211
1,graph_powerlaw_cluster_graph_n7,1251,89.847572,0.071821,0.041961
2,star_graph_n7,9,0.313155,0.034795,0.035996


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,462,38.721584,0.083813,0.051952
1,graph_powerlaw_cluster_graph_n7,1.0,362,26.924842,0.074378,0.010687
2,graph_powerlaw_cluster_graph_n7,3.0,272,18.017185,0.066240,0.065641
3,graph_powerlaw_cluster_graph_n7,4.0,132,5.419997,0.041061,0.041821
4,graph_powerlaw_cluster_graph_n7,0.0,99,1.341304,0.013549,0.001005
5,graph_powerlaw_cluster_graph_n7,5.0,23,0.763964,0.033216,0.054282
6,graph_random_regular_graph_n7_d4,3.0,1389,89.314205,0.064301,0.062061
7,graph_random_regular_graph_n7_d4,2.0,1216,88.504274,0.072783,0.037124
8,graph_random_regular_graph_n7_d4,1.0,582,34.148574,0.058675,0.006071
9,graph_random_regular_graph_n7_d4,4.0,558,22.870940,0.040987,0.042171
